In [1]:
info = """
LangChain 1.0 - RAG Basics (RAG 基础)
=====================================

本模块重点讲解：
1. 文档加载 (Document Loaders)
2. 文本分割 (Text Splitters)
3. 向量嵌入 (Embeddings)
4. 向量存储 (Vector Stores) - Pinecone 免费版
5. 检索 (Retrieval)
6. RAG 问答链
"""
print(info)


LangChain 1.0 - RAG Basics (RAG 基础)

本模块重点讲解：
1. 文档加载 (Document Loaders)
2. 文本分割 (Text Splitters)
3. 向量嵌入 (Embeddings)
4. 向量存储 (Vector Stores) - Pinecone 免费版
5. 检索 (Retrieval)
6. RAG 问答链



In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain.tools import tool
from pinecone import Pinecone, ServerlessSpec
import time

In [8]:
# 获取脚本所在目录
SCRIPT_DIR = os.getcwd()
DATA_DIR = Path.cwd() / "data"


In [9]:
DATA_DIR.mkdir(exist_ok=True)  # exist_ok=True 表示目录已存在时不报错

In [10]:
DATA_DIR

PosixPath('/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data')

In [20]:
# 加载环境变量
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

if not GROQ_API_KEY or GROQ_API_KEY == "your_groq_api_key_here":
    raise ValueError(
        "\n请先在 .env 文件中设置有效的 GROQ_API_KEY\n"
        "访问 https://console.groq.com/keys 获取免费密钥"
    )


In [21]:
model = init_chat_model("groq:llama-3.3-70b-versatile", api_key=GROQ_API_KEY)


In [22]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [24]:
if not PINECONE_API_KEY or PINECONE_API_KEY == "your_pinecone_api_key_here":
    print("\n[警告] 未设置 PINECONE_API_KEY")
    print("如需运行 Pinecone 相关示例，请：")
    print("1. 访问 https://www.pinecone.io/ 注册免费账号")
    print("2. 获取 API Key")
    print("3. 在 .env 文件中设置 PINECONE_API_KEY=你的key")
    print("\n当前将跳过需要 Pinecone 的示例\n")

In [25]:
print( """
    示例1：文档加载

    Document Loaders 将各种数据源转换为 LangChain Document 对象
    """)


    示例1：文档加载

    Document Loaders 将各种数据源转换为 LangChain Document 对象
    


In [26]:
print("\n" + "="*70)
print("示例 1：文档加载 - Document Loaders")
print("="*70)


示例 1：文档加载 - Document Loaders


In [27]:
# 创建示例文本文件
sample_text = """LangChain 是一个用于构建 LLM 应用的框架。

它提供了以下核心组件：
1. Models - 语言模型接口
2. Prompts - 提示词模板
3. Chains - 链式调用
4. Agents - 智能代理
5. Memory - 记忆管理

LangChain 1.0 引入了重大改进，包括：
- 更简洁的 API
- 更好的性能
- 内置的 LangGraph 支持
- 强大的中间件系统

RAG (Retrieval-Augmented Generation) 是 LangChain 的核心应用场景之一。
它结合了检索和生成，让 LLM 能够访问外部知识库。"""

In [28]:
# 保存到文件
doc_path = DATA_DIR / "langchain_intro.txt"
with open(doc_path, "w") as f:
    f.write(sample_text)


In [29]:
# 使用 TextLoader 加载文本文件
loader = TextLoader(doc_path, encoding="utf8")

In [43]:
documents = loader.load()

In [44]:
documents

[Document(metadata={'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}, page_content='LangChain 是一个用于构建 LLM 应用的框架。\n\n它提供了以下核心组件：\n1. Models - 语言模型接口\n2. Prompts - 提示词模板\n3. Chains - 链式调用\n4. Agents - 智能代理\n5. Memory - 记忆管理\n\nLangChain 1.0 引入了重大改进，包括：\n- 更简洁的 API\n- 更好的性能\n- 内置的 LangGraph 支持\n- 强大的中间件系统\n\nRAG (Retrieval-Augmented Generation) 是 LangChain 的核心应用场景之一。\n它结合了检索和生成，让 LLM 能够访问外部知识库。')]

In [36]:
print(f"\n加载结果:")
print(f"  文档数量: {len(documents)}")
print(f"  第一个文档:")
print(f"    内容长度: {len(documents[0].page_content)} 字符")
print(f"    元数据: {documents[0].metadata}")
print(f"    内容预览: {documents[0].page_content[:100]}...")




加载结果:
  文档数量: 1
  第一个文档:
    内容长度: 294 字符
    元数据: {'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}
    内容预览: LangChain 是一个用于构建 LLM 应用的框架。

它提供了以下核心组件：
1. Models - 语言模型接口
2. Prompts - 提示词模板
3. Chains - 链式调用
4. ...


In [37]:
print("\n关键点:")
print("  - TextLoader 加载文本文件")
print("  - 返回 Document 对象列表")
print("  - Document 包含 page_content 和 metadata")
print("\n其他常用 Loaders:")
print("  - PyPDFLoader - 加载 PDF")
print("  - WebBaseLoader - 爬取网页")
print("  - CSVLoader - 加载 CSV")


关键点:
  - TextLoader 加载文本文件
  - 返回 Document 对象列表
  - Document 包含 page_content 和 metadata

其他常用 Loaders:
  - PyPDFLoader - 加载 PDF
  - WebBaseLoader - 爬取网页
  - CSVLoader - 加载 CSV


In [38]:
print( """
    示例2：文本分割

    将长文档分割成小块，便于嵌入和检索
    """)


    示例2：文本分割

    将长文档分割成小块，便于嵌入和检索
    


In [39]:
print("\n" + "="*70)
print("示例 2：文本分割 - Text Splitters")
print("="*70)


示例 2：文本分割 - Text Splitters


In [40]:
# 创建分割器
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", "。", "！", "？", " ", ""]  # 分割优先级
)


In [41]:
print("\n配置:")
print(f"  chunk_size: 200 字符")
print(f"  chunk_overlap: 50 字符（防止信息被截断）")
print(f"  分割优先级: 段落 -> 行 -> 句子 -> 空格 -> 字符")


配置:
  chunk_size: 200 字符
  chunk_overlap: 50 字符（防止信息被截断）
  分割优先级: 段落 -> 行 -> 句子 -> 空格 -> 字符


In [46]:
chunks = splitter.split_documents(documents)

In [47]:
chunks

[Document(metadata={'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}, page_content='LangChain 是一个用于构建 LLM 应用的框架。\n\n它提供了以下核心组件：\n1. Models - 语言模型接口\n2. Prompts - 提示词模板\n3. Chains - 链式调用\n4. Agents - 智能代理\n5. Memory - 记忆管理'),
 Document(metadata={'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}, page_content='LangChain 1.0 引入了重大改进，包括：\n- 更简洁的 API\n- 更好的性能\n- 内置的 LangGraph 支持\n- 强大的中间件系统\n\nRAG (Retrieval-Augmented Generation) 是 LangChain 的核心应用场景之一。\n它结合了检索和生成，让 LLM 能够访问外部知识库。')]

In [48]:
print(f"\n分割结果:")
print(f"  原文档数: {len(documents)}")
print(f"  分割后: {len(chunks)} 块")


分割结果:
  原文档数: 1
  分割后: 2 块


In [49]:
print(f"\n前 3 块内容:")
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\n  块 {i}:")
    print(f"    长度: {len(chunk.page_content)} 字符")
    print(f"    内容: {chunk.page_content[:80]}...")


前 3 块内容:

  块 1:
    长度: 130 字符
    内容: LangChain 是一个用于构建 LLM 应用的框架。

它提供了以下核心组件：
1. Models - 语言模型接口
2. Prompts - 提示词模板
...

  块 2:
    长度: 162 字符
    内容: LangChain 1.0 引入了重大改进，包括：
- 更简洁的 API
- 更好的性能
- 内置的 LangGraph 支持
- 强大的中间件系统

RAG ...


In [50]:
print("\n关键点:")
print("  - chunk_size 控制块大小")
print("  - chunk_overlap 防止信息被截断")
print("  - separators 定义分割优先级")
print("  - RecursiveCharacterTextSplitter 智能分割")


关键点:
  - chunk_size 控制块大小
  - chunk_overlap 防止信息被截断
  - separators 定义分割优先级
  - RecursiveCharacterTextSplitter 智能分割


In [51]:
print("""
    示例3：向量嵌入

    将文本转换为向量，用于相似度计算
    """)


    示例3：向量嵌入

    将文本转换为向量，用于相似度计算
    


In [52]:
print("\n" + "="*70)
print("示例 3：向量嵌入 - Embeddings")
print("="*70)


示例 3：向量嵌入 - Embeddings


In [53]:
print("\n使用 HuggingFace 免费模型:")
print("  模型: sentence-transformers/all-MiniLM-L6-v2")
print("  维度: 384")
print("  特点: 小巧、快速、免费")


使用 HuggingFace 免费模型:
  模型: sentence-transformers/all-MiniLM-L6-v2
  维度: 384
  特点: 小巧、快速、免费


In [56]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [57]:
# 嵌入单个文本
text = "LangChain 是一个 LLM 应用框架"
vector = embeddings.embed_query(text)

In [58]:
vector

[0.06574853509664536,
 0.06645229458808899,
 0.08259230107069016,
 -0.02501906454563141,
 0.006829613819718361,
 0.030552709475159645,
 0.13691017031669617,
 0.022517645731568336,
 0.06587495654821396,
 -0.023874683305621147,
 0.11645320802927017,
 -0.09132852405309677,
 0.027225634083151817,
 -0.04334455728530884,
 0.0017248002113774419,
 0.05244002491235733,
 -0.053088679909706116,
 0.07986626029014587,
 -0.039822887629270554,
 -0.028262071311473846,
 -0.03628155589103699,
 -0.050586991012096405,
 0.06735904514789581,
 0.11166641116142273,
 -0.03474469482898712,
 0.01207928266376257,
 0.05892529338598251,
 0.04497050121426582,
 0.07809198647737503,
 -0.03941391780972481,
 -0.02196110412478447,
 0.06537073850631714,
 0.009936698712408543,
 0.010510641150176525,
 0.00830577127635479,
 0.06580998003482819,
 -0.05630450323224068,
 -0.05059712380170822,
 0.019462477415800095,
 0.015003825537860394,
 -0.02075968310236931,
 -0.0469566285610199,
 0.0669391080737114,
 -0.07444534450769424,
 0

In [59]:
print(f"\n嵌入示例:")
print(f"  文本: {text}")
print(f"  向量维度: {len(vector)}")
print(f"  向量前 5 个值: {vector[:5]}")



嵌入示例:
  文本: LangChain 是一个 LLM 应用框架
  向量维度: 384
  向量前 5 个值: [0.06574853509664536, 0.06645229458808899, 0.08259230107069016, -0.02501906454563141, 0.006829613819718361]


In [60]:
# 嵌入多个文本
texts = [
    "LangChain 是一个框架",
    "Python 是一种编程语言",
    "LangChain 用于构建 LLM 应用"
]

In [61]:
vectors = embeddings.embed_documents(texts)

In [62]:
print(f"\n批量嵌入:")
print(f"  文本数: {len(texts)}")
print(f"  向量数: {len(vectors)}")
print(f"  每个向量维度: {len(vectors[0])}")


批量嵌入:
  文本数: 3
  向量数: 3
  每个向量维度: 384


In [63]:
# 计算相似度（简单示例）
import numpy as np

In [64]:
def cosine_similarity(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [65]:
sim_01 = cosine_similarity(vectors[0], vectors[1])
sim_02 = cosine_similarity(vectors[0], vectors[2])

In [66]:
print(f"\n相似度计算:")
print(f"  '{texts[0]}' vs '{texts[1]}': {sim_01:.4f}")
print(f"  '{texts[0]}' vs '{texts[2]}': {sim_02:.4f}")
print(f"  -> 相同主题的文本相似度更高")



相似度计算:
  'LangChain 是一个框架' vs 'Python 是一种编程语言': 0.6504
  'LangChain 是一个框架' vs 'LangChain 用于构建 LLM 应用': 0.7605
  -> 相同主题的文本相似度更高


In [67]:
print("\n关键点:")
print("  - embed_query() - 嵌入单个查询")
print("  - embed_documents() - 批量嵌入文档")
print("  - 使用免费的 HuggingFace 模型（无需 API key）")
print("  - 向量可用于相似度搜索")


关键点:
  - embed_query() - 嵌入单个查询
  - embed_documents() - 批量嵌入文档
  - 使用免费的 HuggingFace 模型（无需 API key）
  - 向量可用于相似度搜索


In [68]:
print("""
    示例4：Pinecone 设置

    创建 Pinecone serverless 索引（免费层级）
    """)


    示例4：Pinecone 设置

    创建 Pinecone serverless 索引（免费层级）
    


In [69]:
print("\n" + "="*70)
print("示例 4：Pinecone 向量存储 - 创建索引")
print("="*70)


示例 4：Pinecone 向量存储 - 创建索引


In [71]:
if not PINECONE_API_KEY or PINECONE_API_KEY == "your_pinecone_api_key_here":
    print("\n[警告] 跳过：需要设置 PINECONE_API_KEY")


In [72]:
# 初始化 Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [73]:
# 索引配置
index_name = "langchain-rag-demo"
dimension = 384  # 与 all-MiniLM-L6-v2 模型维度匹配

In [74]:
print(f"\n索引配置:")
print(f"  名称: {index_name}")
print(f"  维度: {dimension}")
print(f"  类型: Serverless (免费层级)")
print(f"  区域: us-east-1 (AWS)")


索引配置:
  名称: langchain-rag-demo
  维度: 384
  类型: Serverless (免费层级)
  区域: us-east-1 (AWS)


In [75]:
pc.list_indexes()

[]

In [81]:
# 检查索引是否存在
existing_indexes = [idx.name for idx in pc.list_indexes()]

In [82]:
if index_name in existing_indexes:
    print(f"\n[OK] 索引已存在，直接使用")
    index = pc.Index(index_name)
else:
    print(f"\n创建新索引...")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric="cosine",  # 相似度度量
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"  # 免费层级可用区域
        )
    )

    # 等待索引就绪
    print("等待索引初始化...")
    time.sleep(10)
    index = pc.Index(index_name)
    print("[OK] 索引创建完成")


[OK] 索引已存在，直接使用


In [83]:
# 获取索引统计
stats = index.describe_index_stats()
print(f"\n索引统计:")
print(f"  向量数: {stats.get('total_vector_count', 0)}")
print(f"  维度: {stats.get('dimension', 'N/A')}")


索引统计:
  向量数: 0
  维度: 384


In [80]:
print("\n关键点:")
print("  - Pinecone 提供免费 serverless 层级")
print("  - dimension 必须与 embedding 模型匹配")
print("  - metric='cosine' 用于相似度计算")
print("  - ServerlessSpec 配置云和区域")


关键点:
  - Pinecone 提供免费 serverless 层级
  - dimension 必须与 embedding 模型匹配
  - metric='cosine' 用于相似度计算
  - ServerlessSpec 配置云和区域


In [84]:
print("""
    示例5：文档索引

    将分割后的文档存入 Pinecone
    """
    )


    示例5：文档索引

    将分割后的文档存入 Pinecone
    


In [85]:
print("\n" + "="*70)
print("示例 5：文档索引 - 存入向量数据库")
print("="*70)



示例 5：文档索引 - 存入向量数据库


In [86]:
if not index_name or not embeddings:
    print("\n[警告] 跳过：需要 Pinecone 配置")

In [87]:
print(f"\n准备索引 {len(chunks)} 个文档块...")



准备索引 2 个文档块...


In [88]:
# 使用 PineconeVectorStore.from_documents()
vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embeddings,
    index_name=index_name
)

In [89]:
print(f"[OK] 文档已索引到 Pinecone")


[OK] 文档已索引到 Pinecone


In [90]:
# 测试检索
query = "LangChain 的核心组件是什么？"
print(f"\n测试检索:")
print(f"  查询: {query}")



测试检索:
  查询: LangChain 的核心组件是什么？


In [91]:
results = vectorstore.similarity_search(query, k=2)
results

[Document(id='bc00d2ee-fd76-4d0e-b600-77a1346be8de', metadata={'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}, page_content='LangChain 是一个用于构建 LLM 应用的框架。\n\n它提供了以下核心组件：\n1. Models - 语言模型接口\n2. Prompts - 提示词模板\n3. Chains - 链式调用\n4. Agents - 智能代理\n5. Memory - 记忆管理'),
 Document(id='8fd58aff-b1fe-4609-89f7-22bca9fdabed', metadata={'source': '/Users/liujiewei/DevEnv/codeReg/python/Langchain1.0-Langgraph1.0-Learning/phase2_practical/13_rag_basics/data/langchain_intro.txt'}, page_content='LangChain 1.0 引入了重大改进，包括：\n- 更简洁的 API\n- 更好的性能\n- 内置的 LangGraph 支持\n- 强大的中间件系统\n\nRAG (Retrieval-Augmented Generation) 是 LangChain 的核心应用场景之一。\n它结合了检索和生成，让 LLM 能够访问外部知识库。')]

In [92]:
print(f"  返回 {len(results)} 个最相关的文档块:\n")
for i, doc in enumerate(results, 1):
    print(f"  结果 {i}:")
    print(f"    内容: {doc.page_content[:100]}...")
    print()

  返回 2 个最相关的文档块:

  结果 1:
    内容: LangChain 是一个用于构建 LLM 应用的框架。

它提供了以下核心组件：
1. Models - 语言模型接口
2. Prompts - 提示词模板
3. Chains - 链式调用
4. ...

  结果 2:
    内容: LangChain 1.0 引入了重大改进，包括：
- 更简洁的 API
- 更好的性能
- 内置的 LangGraph 支持
- 强大的中间件系统

RAG (Retrieval-Augmented...



In [93]:
print("关键点:")
print("  - from_documents() 自动嵌入并存储")
print("  - similarity_search() 检索相似文档")
print("  - k=2 返回最相关的 2 个结果")

关键点:
  - from_documents() 自动嵌入并存储
  - similarity_search() 检索相似文档
  - k=2 返回最相关的 2 个结果


In [94]:
print( """
    示例6：RAG 问答

    将向量存储转为工具，供 Agent 使用
    """)
    


    示例6：RAG 问答

    将向量存储转为工具，供 Agent 使用
    


In [95]:
print("\n" + "="*70)
print("示例 6：RAG 问答 - 使用检索工具")
print("="*70)


示例 6：RAG 问答 - 使用检索工具


In [96]:
if not vectorstore:
    print("\n[警告] 跳过：需要 Pinecone vectorstore")

In [97]:
# 创建检索工具
@tool
def search_knowledge_base(query: str) -> str:
    """在知识库中搜索相关信息"""
    docs = vectorstore.similarity_search(query, k=3)
    return "\n\n".join([doc.page_content for doc in docs])

In [98]:
# 创建 Agent
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[search_knowledge_base],
    system_prompt="""你是一个助手，可以访问知识库。
当用户提问时：
1. 使用 search_knowledge_base 工具搜索相关信息
2. 基于搜索结果回答问题
3. 如果知识库中没有相关信息，诚实告知"""
)


In [99]:
# 测试问答
questions = [
    "LangChain 有哪些核心组件？",
    "RAG 是什么？",
    "LangChain 1.0 有什么改进？"
]

In [100]:
for question in questions:
    print(f"\n问题: {question}")
    try:
        response = agent.invoke({"messages": [{"role": "user", "content": question}]})
        print(f"回答: {response['messages'][-1].content}")
    except Exception as e:
        print(f"[错误] 查询失败: {str(e)[:100]}...")
        print("提示: 这可能是 Groq 模型处理中文工具调用的偶发问题")
        print("解决方案: 1) 重试 2) 使用英文提问 3) 换用其他模型")
    print("-" * 70)


问题: LangChain 有哪些核心组件？
回答: LangChain 是一个用于构建 LLM 应用的框架。它提供了以下核心组件：
1. Models - 语言模型接口
2. Prompts - 提示词模板
3. Chains - 链式调用
4. Agents - 智能代理
5. Memory - 记忆管理

LangChain 1.0 引入了重大改进，包括：
- 更简洁的 API
- 更好的性能
- 内置的 LangGraph 支持
- 强大的中间件系统

RAG (Retrieval-Augmented Generation) 是 LangChain 的核心应用场景之一。
它结合了检索和生成，让 LLM 能够访问外部知识库。
----------------------------------------------------------------------

问题: RAG 是什么？
回答: RAG 的全称是 Retrieval-Augmented Generation，它是一种结合检索和生成的技术，用于提高大型语言模型（LLM）的性能和知识获取能力。RAG 允许模型从外部知识库中检索信息并将其整合到生成的文本中，从而使其能够更好地回答问题和生成文本。
----------------------------------------------------------------------

问题: LangChain 1.0 有什么改进？
回答: LangChain 1.0 的改进包括更简洁的 API、更好的性能、内置的 LangGraph 支持和强大的中间件系统。
----------------------------------------------------------------------


In [101]:
print("\n关键点:")
print("  - 将 vectorstore 封装为工具")
print("  - Agent 自动调用工具检索")
print("  - 基于检索结果生成答案")
print("  - 这就是 RAG (检索增强生成)")



关键点:
  - 将 vectorstore 封装为工具
  - Agent 自动调用工具检索
  - 基于检索结果生成答案
  - 这就是 RAG (检索增强生成)
